In [1]:
from pandas_datareader import data as pdr
import datetime
import yfinance as yfin

yfin.pdr_override()

start = "2023-01-1"
end = "2023-07-31"

amzn = pdr.get_data_yahoo("AMZN",start, end)
googl = pdr.get_data_yahoo("GOOGL", start, end)

[*********************100%***********************]  1 of 1 completed
